In [1]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time
import sys

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf

from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
current_dir = os.getcwd()
source_dir = os.path.join(current_dir)
first_language = 'pl'
second_language = 'en'

source_dir = os.path.join(source_dir, f'{first_language}_{second_language}')

bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    vocab_size = 8000,
    reserved_tokens=reserved_tokens,
    bert_tokenizer_params=bert_tokenizer_params,
    learn_params={},
)

START = tf.argmax(tf.constant(reserved_tokens) == "[START]")
END = tf.argmax(tf.constant(reserved_tokens) == "[END]")

def add_start_end(ragged):
  count = ragged.bounding_shape()[0]
  starts = tf.fill([count,1], START)
  ends = tf.fill([count,1], END)
  return tf.concat([starts, ragged, ends], axis=1)

def cleanup_text(reserved_tokens, token_txt):
  bad_tokens = [re.escape(tok) for tok in reserved_tokens if tok != "[UNK]"]
  bad_token_re = "|".join(bad_tokens)

  bad_cells = tf.strings.regex_full_match(token_txt, bad_token_re)
  result = tf.ragged.boolean_mask(token_txt, ~bad_cells)

  result = tf.strings.reduce_join(result, separator=' ', axis=-1)

  return result

class CustomTokenizer(tf.Module):
  def __init__(self, reserved_tokens, vocab_path):
    self.tokenizer = text.BertTokenizer(vocab_path, lower_case=True)
    self._reserved_tokens = reserved_tokens
    self._vocab_path = tf.saved_model.Asset(vocab_path)

    vocab = pathlib.Path(vocab_path).read_text(encoding='utf-8').splitlines()
    self.vocab = tf.Variable(vocab)

    self.tokenize.get_concrete_function(
        tf.TensorSpec(shape=[None], dtype=tf.string))

    self.detokenize.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.detokenize.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    self.lookup.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.lookup.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    self.get_vocab_size.get_concrete_function()
    self.get_vocab_path.get_concrete_function()
    self.get_reserved_tokens.get_concrete_function()

  @tf.function
  def tokenize(self, strings):
    enc = self.tokenizer.tokenize(strings)
    enc = enc.merge_dims(-2,-1)
    enc = add_start_end(enc)
    return enc

  @tf.function
  def detokenize(self, tokenized):
    words = self.tokenizer.detokenize(tokenized)
    return cleanup_text(self._reserved_tokens, words)

  @tf.function
  def lookup(self, token_ids):
    return tf.gather(self.vocab, token_ids)

  @tf.function
  def get_vocab_size(self):
    return tf.shape(self.vocab)[0]

  @tf.function
  def get_vocab_path(self):
    return self._vocab_path

  @tf.function
  def get_reserved_tokens(self):
    return tf.constant(self._reserved_tokens)


def write_vocab_file(filepath, vocab):
  with open(filepath, 'w',encoding='utf-8') as f:
    for token in vocab:
      print(token, file=f)
      
def make_own_tokenizers(train_first, train_secound, first_language='',secound_language=''):
    first_vocab = bert_vocab.bert_vocab_from_dataset(
        train_first.batch(1000).prefetch(2),
        **bert_vocab_args
    )

    secound_vocab = bert_vocab.bert_vocab_from_dataset(
        train_secound.batch(1000).prefetch(2),
        **bert_vocab_args
    )

    write_vocab_file(os.path.join(source_dir,f'{first_language}_vocab.txt'), first_vocab)
    write_vocab_file(os.path.join(source_dir,f'{secound_language}_vocab.txt'), secound_vocab)

    first_tokenizer = text.BertTokenizer(os.path.join(source_dir,f'{first_language}_vocab.txt'), **bert_tokenizer_params)
    secound_tokenizer = text.BertTokenizer(os.path.join(source_dir,f'{secound_language}_vocab.txt'), **bert_tokenizer_params)

    return first_tokenizer, secound_tokenizer

In [5]:
df = pd.read_csv(os.path.join(source_dir,f'{first_language}_{second_language}.tsv'), sep='\t', header=None, names=[f'ID_{first_language}', first_language,f'ID_{second_language}', second_language]).loc[:, [first_language, second_language]]

train_examples, val_examples = train_test_split(df, test_size=0.05, random_state=42)

train_pl = train_examples[[first_language]]
train_en = train_examples[[second_language]]

train_pl = tf.data.Dataset.from_tensor_slices(train_pl.values)
train_en = tf.data.Dataset.from_tensor_slices(train_en.values)

make_own_tokenizers(train_pl, train_en, first_language=first_language, secound_language=second_language)

(<tensorflow_text.python.ops.bert_tokenizer.BertTokenizer at 0x190e1b917b0>,
 <tensorflow_text.python.ops.bert_tokenizer.BertTokenizer at 0x190dba8f6a0>)

In [8]:
model_name_pl_en = os.path.join(source_dir, f'translate_{first_language}_{second_language}_converter')

tokenizers = tf.Module()
tokenizers.pl= CustomTokenizer(reserved_tokens, os.path.join(source_dir,f'{first_language}_vocab.txt'))
tokenizers.en = CustomTokenizer(reserved_tokens, os.path.join(source_dir,f'{second_language}_vocab.txt'))

tf.saved_model.save(tokenizers, model_name_pl_en)

INFO:tensorflow:Assets written to: d:\Folder\Studia\VIII Semestr\Sztuczne sieci neuronowe\SSN_Example\pl_en\translate_pl_en_converter\assets


In [9]:
reloaded_tokenizers = tf.saved_model.load(model_name_pl_en)
reloaded_tokenizers.en.get_vocab_size().numpy()

tokens = reloaded_tokenizers.en.tokenize(['Let s try something.'])

round_trip = reloaded_tokenizers.en.detokenize(tokens)

print(round_trip.numpy()[0].decode('utf-8'))

reloaded_tokenizers.pl.get_vocab_size().numpy()

tokens = reloaded_tokenizers.pl.tokenize(['Spróbujmy coś.'])

round_trip = reloaded_tokenizers.pl.detokenize(tokens)

print(round_trip.numpy()[0].decode('utf-8'))

let s try something .
sprobujmy cos .
